In [2]:
import tensorflow as tf
import os
print(tf.__version__)

2.4.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers
!pip install bert
import sys
sys.path.append('/content/drive/MyDrive/ner')
from OurBertModel import MyBertModel
from OurBertModelTrainer import BertModelTrainer
from transformers import AutoTokenizer

     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 890kB 18.4MB/s 
     |████████████████████████████████| 2.9MB 17.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1ab784d7efd481070c7ad0bc5fce9046e36c130e5ddfacfa8b77292a3ccc86f6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for bert: filename=bert-2.2.0-cp36-none-any.whl size=3756 sha256=3ec1bba86bcf52de57c00df619372d13483b941c00c8d96e083e7a74fcc092b2
  Stored in directory: /root/.cache/pip/wheels/fe/71/b7/941459453bd38e5d97a8c886361dee19325e9933c9cf88ad46
  Created wheel for erlastic: filename=erlastic-2.0.0-cp36-none-any.whl size=6787 sha256=e1e9391b1e85c0ea7c6dd44ee85b350009e6a5773904ceb2a811a821d700d257
  Stored in directory: /root/.cache/pip/wheels/02/62/46/93c713a5f061aeeb4f16eb6bf5ee798816e6ddda70faa78e69
Successfully built be

In [4]:
import pandas as pd
import math
import numpy as np
#from seqeval.metrics import f1_score
#import seqeval
import time
import re

#from seqeval.metrics import classification_report,accuracy_score,f1_score
from tqdm import tqdm,trange

In [5]:
df_data = pd.read_csv("/content/drive/MyDrive/ner/NER2.csv",sep="\t").fillna(method='ffill')
#df_data.shape

In [6]:
df_data.columns = ['class','tag','sentence']
#df_data.head()

In [8]:
Slice = 300000
df_data = df_data[:Slice]

In [9]:
def turn_lower(sent):
    return sent.lower()
def removePunct(sent):
    sent = re.sub(r'_', '', sent)
    return re.sub(r'[^\w\s]', '', sent)

df_data.sentence = df_data.sentence.apply(turn_lower)
df_data.sentence = df_data.sentence.apply(removePunct)
df_data.head()

,class,tag,sentence
0,law,B-MISC O B-ORGANIZATION I-ORGANIZATION I-ORGAN...,casanova isviçre federal yüksek mahkemesi esk...
1,government,B-PERSON I-PERSON O B-MISC O O,corina casanova bir federal parlementerdir
2,location,B-PERSON O O O O O O O O O O O O O O O B-MISC ...,casanova hristiyan demokrat halk partisi üyes...
3,government,B-ORGANIZATION I-ORGANIZATION I-ORGANIZATION O...,isviçre dışişleri bakanlığı isviçre federal y...
4,government,B-LOCATION O O O O O,isviçrenin dış ilişkilerini sürdürmekle görevl...


In [10]:
df_data.sentence[0].split()

['casanova',
 'isviçre',
 'federal',
 'yüksek',
 'mahkemesi',
 'eski',
 'başkanı',
 'nay',
 'giusepin',
 'pratiğinde',
 'bir',
 'avukat',
 'olarak',
 'çalıştı']

In [11]:
tag = df_data.tag[0].split()
word = df_data.sentence[0].split()
label = [(w,t) for t,w in zip(tag,word)]
label

[('casanova', 'B-MISC'),
 ('isviçre', 'O'),
 ('federal', 'B-ORGANIZATION'),
 ('yüksek', 'I-ORGANIZATION'),
 ('mahkemesi', 'I-ORGANIZATION'),
 ('eski', 'I-ORGANIZATION'),
 ('başkanı', 'O'),
 ('nay', 'O'),
 ('giusepin', 'O'),
 ('pratiğinde', 'O'),
 ('bir', 'O'),
 ('avukat', 'O'),
 ('olarak', 'O'),
 ('çalıştı', 'B-MISC')]

In [12]:
def createTagWord(data):
    label_list = []
    for i in range(len(data)):
        tag = data.tag[i].split()
        word = data.sentence[i].split()
        label_list.append([(w,t) for t,w in zip(tag,word)])
    return label_list

In [13]:
label_word_list = createTagWord(df_data)
label_word_list[5]

[('gilgit', 'B-LOCATION'),
 ('baltistan', 'I-LOCATION'),
 ('veya', 'O'),
 ('pakistan', 'O'),
 ('kuzey', 'B-LOCATION'),
 ('bölgeler', 'O'),
 ('urduca', 'O'),
 ('gilgit', 'O'),
 ('baltistn', 'B-MISC'),
 ('balti', 'O'),
 ('dili', 'B-LOCATION'),
 ('keşmir', 'O'),
 ('bölgesinin', 'O'),
 ('kuzey', 'O'),
 ('kesiminde', 'O'),
 ('pakistan', 'O'),
 ('sınırları', 'O'),
 ('içinde', 'O'),
 ('özerk', 'O'),
 ('bir', 'O'),
 ('statüye', 'O'),
 ('sahip', 'B-LOCATION'),
 ('pakistanın', 'O'),
 ('keşmir', 'O'),
 ('bölgesindeki', 'O'),
 ('federal', 'O'),
 ('idari', 'O'),
 ('yönetim', 'O'),
 ('bölgesi', 'B-LOCATION')]

In [15]:
"""
# if csv tag&id.csv is exist, dont run this block
tags = df_data.tag.values
tags = " ".join(tags)
tags = tags.split()
tags = set(tags)
tags2index = {t:i+1 for i,t in enumerate(tags)}
tag = []
number = []
for t in tags2index:
    tag.append(t)
    number.append(tags2index[t])

tag = pd.DataFrame(tag)
number = pd.DataFrame(number)
tags2index_df = pd.concat([tag,number],axis=1)
tags2index_df.to_csv("tag&id.csv")
"""

'\n# if csv tag&id.csv is exist, dont run this block\ntags = df_data.tag.values\ntags = " ".join(tags)\ntags = tags.split()\ntags = set(tags)\ntags2index = {t:i+1 for i,t in enumerate(tags)}\ntag = []\nnumber = []\nfor t in tags2index:\n    tag.append(t)\n    number.append(tags2index[t])\n\ntag = pd.DataFrame(tag)\nnumber = pd.DataFrame(number)\ntags2index_df = pd.concat([tag,number],axis=1)\ntags2index_df.to_csv("tag&id.csv")\n'

In [14]:
convert_int = lambda x: int(x)

tags2index = pd.read_csv("/content/drive/MyDrive/ner/tag&id.csv",names=['tag','ids'],skiprows=1)
tag = tags2index.tag
ids = tags2index.ids.apply(convert_int)
tags2index = {t:i for t,i in zip(tag,ids)}
tags2index

{'B-LOCATION': 3,
 'B-MISC': 6,
 'B-ORGANIZATION': 9,
 'B-PERSON': 1,
 'I-LOCATION': 7,
 'I-MISC': 8,
 'I-ORGANIZATION': 2,
 'I-PERSON': 4,
 'O': 5}

In [15]:
tag_id = [[tags2index[w[1]] for w in s] for s in label_word_list]
tag_id[0]


[6, 5, 9, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 6]

In [16]:
from tensorflow.compat.v1.keras.preprocessing.sequence import pad_sequences
#from keras.preprocessing.sequence import pad_sequences
max_len = 50
tag_id = [[tags2index[w[1]] for w in s] for s in label_word_list]
tag_id_pad = pad_sequences(maxlen=max_len, sequences=tag_id, padding="post", value=tags2index["O"])
tag_id_pad[0]

array([6, 5, 9, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5], dtype=int32)

In [17]:
tag_id[8],label_word_list[8]

([6, 5, 6, 5, 6, 5, 5, 5, 5, 5],
 [('urduca', 'B-MISC'),
  ('peştuca', 'O'),
  ('wakhi', 'B-MISC'),
  ('shina', 'O'),
  ('balti', 'B-MISC'),
  ('khuar', 'O'),
  ('ve', 'O'),
  ('buruşaski', 'O'),
  ('dilleri', 'O'),
  ('konuşulmaktadır', 'O')])

In [20]:
"""
from tensorflow.compat.v1.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
word_list = []
for sent in pad_sentences:
    for word in sent:
        if word != "[PAD]":
            word_list.append(word)
words = " ".join(word_list)
tokenizer.fit_on_texts(words.split())

token_to_word = dict(zip(tokenizer.word_index.values(), tokenizer.word_index.keys()))
word_to_token = dict(zip(tokenizer.word_index.keys(), tokenizer.word_index.values()))

token_to_word[0] = "[PAD]"
word_to_token["[PAD]"] = 0
"""

'\nfrom tensorflow.compat.v1.keras.preprocessing.text import Tokenizer\ntokenizer = Tokenizer()\nword_list = []\nfor sent in pad_sentences:\n    for word in sent:\n        if word != "[PAD]":\n            word_list.append(word)\nwords = " ".join(word_list)\ntokenizer.fit_on_texts(words.split())\n\ntoken_to_word = dict(zip(tokenizer.word_index.values(), tokenizer.word_index.keys()))\nword_to_token = dict(zip(tokenizer.word_index.keys(), tokenizer.word_index.values()))\n\ntoken_to_word[0] = "[PAD]"\nword_to_token["[PAD]"] = 0\n'

In [18]:
max_len = 50
sentence_list = [[w[0] for w in s] for s in label_word_list]
pad_sentences = []
for seq in tqdm(sentence_list):
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("[PAD]")
    pad_sentences.append(new_seq)
pad_sentences[0][10:15]

100%|██████████| 300000/300000 [00:06<00:00, 49509.40it/s]


['bir', 'avukat', 'olarak', 'çalıştı', '[PAD]']

In [19]:
turkishTokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')
print(turkishTokenizer.tokenize("sarmal"))
print(turkishTokenizer.encode("sarmal"))
a = turkishTokenizer.encode("sarmal")
print(turkishTokenizer.decode(a))


['sarma', '##l']
[2, 23918, 1008, 3]
[CLS] sarmal [SEP]


In [20]:
turkishTokenizer = AutoTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')
sentence_id = []
for sent in tqdm(pad_sentences):
    word_id = []
    for word in sent:
        if word != "[PAD]":
            word_id.append(turkishTokenizer.encode(word)[1])
        else:
            word_id.append(0)
    sentence_id.append(word_id)
sentence_id[0]

100%|██████████| 300000/300000 [03:56<00:00, 1268.17it/s]


[8512,
 2605,
 21820,
 2862,
 27279,
 3275,
 6331,
 4044,
 3509,
 23871,
 1996,
 7154,
 2133,
 9879,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [21]:
from sklearn.model_selection import train_test_split
X_train_id, X_test_id, y_train, y_test = train_test_split(sentence_id, tag_id_pad, test_size=0.1, random_state=3)    

In [22]:
def return_token_type_ids(input_id):
        segment = []
        s = 1
        for sentence_ids in input_id:
            sentence_segment = []
            for ids in sentence_ids:
                if ids == 0:
                    s=0
                else:
                    s=1   
                sentence_segment.append(s)
  
            segment.append(sentence_segment)
            
        return segment
X_train_segment = return_token_type_ids(X_train_id)
X_test_segment = return_token_type_ids(X_test_id)

In [23]:
print(X_train_id[0])
print(X_train_segment[0])
print(y_train[0])

[25, 21870, 12724, 3863, 8872, 23192, 24186, 3176, 9009, 5388, 8916, 1992, 5829, 8235, 24153, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[5 6 6 5 5 1 4 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5]


In [24]:
X_train_id = np.array(X_train_id)
X_train_segment = np.array(X_train_segment)
y_train = np.array(y_train)

print(X_train_id.shape)
print(X_train_segment.shape)
print(y_train.shape)

(270000, 50)
(270000, 50)
(270000, 50)


In [25]:
NB_ENCODER = 12
FFN_UNITS = 768
NB_ATTENTION_HEAD = 8 #12
HIDDEN_UNITS = 512 #768
VOCAB_SIZE_TURKISH = turkishTokenizer.vocab_size
DROPOUT = 0.10
SEQ_LEN = 64

In [26]:
OUR_BERT = MyBertModel(nb_encoder_layers=NB_ENCODER,
                        FFN_units=FFN_UNITS,
                        nb_attention_head=NB_ATTENTION_HEAD,
                        nb_hidden_units=HIDDEN_UNITS,
                        dropout_rate=DROPOUT,
                        vocab_size=VOCAB_SIZE_TURKISH)

print("Downloading BERT model...")
Trainer = BertModelTrainer(HIDDEN_UNITS) 
MY_BERT =  Trainer(BertModel=OUR_BERT,
                    epochs=0,
                    inputs=[],
                    NSP_label=[],
                    mask_index=[],
                    mask_label=[],
                    segment=[],
                    checkpoint_path="/content/drive/MyDrive/ner/Bert_Checkpoint/",
                    max2keep=2,
                    batch2Show=8)


Latest checkpoint restored!!


In [27]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train_id,X_train_segment,y_train)).shuffle(1000).batch(32).repeat(5)

In [28]:
MY_BERT.layers[-4].activation

<bound method MyBertModel.gelu_activation_function of <OurBertModel.MyBertModel object at 0x7fbeac870f60>>

In [29]:
class myNERmodel(tf.keras.Model):
    def __init__(self,nb_class,name="NER_Model"):
        super(myNERmodel, self).__init__(name=name)
        
        self.BERT_MODEL = MY_BERT
        
        self.last_layer = tf.keras.layers.Dense(units=nb_class,activation="softmax")
        
    def __call__(self,sentence_id,segment_id,training):
        outputs = self.BERT_MODEL(sentence_id,segment_id,training)
        outputs = self.last_layer(outputs)
        
        return outputs

In [30]:
NB_CLASS = len(tags2index)+1
NER_Model = myNERmodel(NB_CLASS)

In [31]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [32]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction="none")
        
train_loss = tf.keras.metrics.Mean(name="train_loss")

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

leaning_rate = 3e-5
optimizer = tf.keras.optimizers.Adam(leaning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98,
                                        epsilon=1e-9)

In [33]:
def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0))
    loss_ = loss_object(target, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

In [34]:
ckpt = tf.train.Checkpoint(NER_Model=NER_Model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, "/content/drive/MyDrive/ner/nercheck/", max_to_keep=2)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [35]:
for id_batch,segment_batch,y_batch in train_ds:
    print(id_batch)
    print(segment_batch)
    print(y_batch)
    break

tf.Tensor(
[[ 5784 21097  2400 ...     0     0     0]
 [ 3007  5788  6861 ...     0     0     0]
 [   82    91 14437 ...     0     0     0]
 ...
 [ 3041 13481  5945 ...     0     0     0]
 [19840 30055  2478 ...     0     0     0]
 [18208 26779 29255 ...     0     0     0]], shape=(32, 50), dtype=int64)
tf.Tensor(
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]], shape=(32, 50), dtype=int64)
tf.Tensor(
[[6 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]
 [6 5 5 ... 5 5 5]
 ...
 [6 5 5 ... 5 5 5]
 [3 5 5 ... 5 5 5]
 [5 5 5 ... 5 5 5]], shape=(32, 50), dtype=int32)


In [36]:
EPOCHS = 30
for epoch in range(EPOCHS):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for (batch, (id_batch,segment_batch,y_batch)) in enumerate(train_ds):
        with tf.GradientTape() as tape:
            predictions = NER_Model(id_batch, segment_batch, True)
            loss = loss_function(y_batch, predictions)
        
        gradients = tape.gradient(loss, NER_Model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, NER_Model.trainable_variables))
        
        train_loss(loss)
        train_accuracy(y_batch, predictions)
        
        if batch % 50 == 0:
            print("Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))
            grad_list = [grad for grad in gradients if grad is not None]
            print("Number of not None grads: {}".format(len(grad_list)))
            print("ALL Trainable Variables:{}".format(len(NER_Model.trainable_variables)))
        if batch == 6000:
            break;
           
    ckpt_save_path = ckpt_manager.save()
    print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                        ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))
    


Start of epoch 1
Epoch 1 Batch 0 Loss 0.1798 Accuracy 0.9513
Number of not None grads: 200
ALL Trainable Variables:200


KeyboardInterrupt: ignored

## TEST

In [153]:
  for (batch, (id_batch,segment_batch,y_batch)) in enumerate(train_ds):
        sentence_in = id_batch[1]
        tag_in = y_batch[1]
        segment_id = segment_batch[1]
        break

In [154]:
y_batch.shape

TensorShape([32, 50])

In [155]:
sentence_id = sentence_in[:10]
tag_id = [tag_in[:10]]
segment_id = segment_id[:10]

print(turkishTokenizer.decode(sentence_id))
print(sentence_id)
print(tag_id)
print(segment_id)

ann kar go tam sekiz filminde oynayacak [PAD] [PAD] [PAD]
tf.Tensor([ 3888  2096  9153  2955  8755 15822 16633     0     0     0], shape=(10,), dtype=int64)
[<tf.Tensor: shape=(10,), dtype=int32, numpy=array([1, 4, 5, 5, 5, 6, 5, 5, 5, 5], dtype=int32)>]
tf.Tensor([1 1 1 1 1 1 1 0 0 0], shape=(10,), dtype=int64)


In [156]:
sentence_id = np.array(sentence_id).reshape(1,-1)
tag_id = np.array(tag_id).reshape(1,-1)
segment_id = np.array(segment_id).reshape(1,-1)

In [157]:
sentence_id.shape

(1, 10)

In [158]:
tag_id

array([[1, 4, 5, 5, 5, 6, 5, 5, 5, 5]], dtype=int32)

In [159]:
predictions = NER_Model(sentence_id, segment_id, False)

In [160]:
tag_id

array([[1, 4, 5, 5, 5, 6, 5, 5, 5, 5]], dtype=int32)

In [1]:
output_index = []
for pred in predictions:
    for embed in pred:
        output_index.append(np.argmax(embed))

NameError: ignored

In [162]:
tag_id,output_index

(array([[1, 4, 5, 5, 5, 6, 5, 5, 5, 5]], dtype=int32),
 [1, 4, 5, 5, 5, 5, 5, 5, 5, 5])

In [163]:
#compare tag and output_index
for t,o in zip(tag_id[0],output_index):
    print("real:{}---pred:{}".format(t,o))

real:1---pred:1
real:4---pred:4
real:5---pred:5
real:5---pred:5
real:5---pred:5
real:6---pred:5
real:5---pred:5
real:5---pred:5
real:5---pred:5
real:5---pred:5


In [164]:
output_index = np.array(output_index).reshape(-1,1)
output_index.shape

(10, 1)

In [165]:
tag_id=tag_id.reshape(-1,1)
tag_id.shape

(10, 1)

In [166]:
train_accuracy_for_eval = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
train_accuracy_for_eval(tag_id, predictions)

<tf.Tensor: shape=(), dtype=float32, numpy=0.58>

In [56]:
tag_id

array([[6],
       [8],
       [5],
       [5],
       [5],
       [5],
       [9],
       [2],
       [5],
       [5]], dtype=int32)

In [ ]:
output_index

array([[5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5]])